<a href="https://colab.research.google.com/github/saspav/Introduction_Competitive_Data_Science/blob/main/tasks/6_2_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача на 3-ом шаге урока

**Застекай их**

Объявите модель StackingClassifier(), которая в качестве базовых моделей будет принимать список estimators, а в качестве метамодели использует LogisticRegression().

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

estimators = [("logreg", LogisticRegression()), ("svc", SVC())]
meta_model = StackingClassifier(estimators, final_estimator=LogisticRegression())

# Задача на 5-ом шаге урока

**Разгоняем метамодель**

Ссылка на данные для тестирования

In [33]:
import numpy as np
import pandas as pd
from itertools import product

df = pd.read_csv('https://stepik.org/media/attachments/lesson/825511/meta_feautres_data.csv')

df.head()

,index,target,model
0,0,65.927922,catboost
1,1,44.248116,catboost
2,2,43.807966,catboost
3,3,45.661121,catboost
4,4,43.653036,catboost


In [34]:
pvt = df.pivot_table(index='index', columns='model', values='target')
pvt_columns = pvt.columns

for (oper, func), pair in product((('dif', pd.Series.sub), ('div', pd.Series.divide)),
                                  product(pvt_columns, repeat=2)):
    pvt[f'_{oper}_'.join(pair)] = func(*[pvt[col] for col in pair])

pvt.rename(columns={col: f'target_{col}' for col in pvt_columns}, inplace=True)
pvt.to_csv('submit_6_2_6_V2.csv')


# Задача на 6-ом и 7-ом шаге урока

**Пишем свой стекер и обучаем его**

На 6-ом шаге урока пишем класс Blender.
На 7-ом шаге урока воспользуйся классом Blender, который написал на предыдущем задании, и обучи свой ансамбль.

Для тестирования на данных можно взять датасет

In [35]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold

X, y = datasets.load_diabetes(return_X_y=True)
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, random_state=42, shuffle=True)

Решение шаг 6

In [36]:
class Blender:
    def __init__(self, base_models, meta_model):
        """
        base_models - список базовых моделей, которые нужно обучать на изначальных данных
        meta_model - мета модель, которая обучается на предсказаниях базовых моделей
        Считайте, что модель, которая передается имеет поля .fit(X_train, y_train) и .predict(X_test)
        .predict(X_test) возвращает предсказания размерности (n_samples,)
        """
        self.base_models = base_models
        self.meta_model = meta_model

    def stack_predicts(self, X):
        return np.hstack([model.predict(X).reshape(-1, 1) for model in self.base_models])

    def fit(self, X_train, y_train, X_hold, y_hold):
        [model.fit(X_train, y_train) for model in self.base_models]
        self.meta_model.fit(self.stack_predicts(X_hold), y_hold)

    def predict(self, X_test):
        return self.meta_model.predict(self.stack_predicts(X_test))

Решение шаг 7

In [37]:
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.svm import SVR

# Теперь обучи свой ансамбль и положи предсказания на X_holdout в переменную preds
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, random_state=42, shuffle=True)

base_models = [LinearRegression(), LassoCV(), SVR()]
meta_model = Blender(base_models, LinearRegression())
meta_model.fit(X_train, y_train, X_holdout, y_holdout)
preds = meta_model.predict(X_holdout)

# Задача на 10-ом шаге урока

**Пишем CV**

Решение

In [38]:
def GetPreds(model, X, y, n_fold=5):
    """
    :param model: имеет методы .fit(X, y) и .predict(X)
    :param X: pd.DataFrame
    :param y: np.array
    :param n_fold: количество фолдов
    :return: np.array
    """
    folds = KFold(n_splits=n_fold)
    preds = np.zeros(X.shape[0])

    for train_idx, valid_idx in folds.split(X, y):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model.fit(X_train, y_train)
        preds[valid_idx] = model.predict(X_valid).flatten()

    return preds.reshape(-1, 1)

Для тестирования функции можно использовать следующий код:

In [39]:
from sklearn import datasets
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR

X, y = datasets.load_diabetes(return_X_y=True)
X = pd.DataFrame(X)

base_models = [LinearRegression(), Lasso(), SVR()]
for model in base_models:
    GetPreds(model, X, y)

# Задача на 11-ом шаге урока

**Добавляем X_test**

Модифицируйте функцию из предыдущей задачи, чтобы она, помимо предсказаний на весь X, еще возвращала предсказания на X_test. Для того чтобы получить предсказания на X_test, обучите данную модель сразу на всей выборке.

Решение

Для тестирования функции можно использовать следующий код:

In [40]:
def GetPreds(model, X, y, X_test, n_fold=5):
    """
    :param model: имеет методы .fit(X, y) и .predict(X)
    :param X: pd.DataFrame
    :param y: np.array
    :param X_test:  pd.DataFrame, для которого нужно сделать предсказания
    :param n_fold: количество фолдов
    :return: np.array, np.array
    """
    folds = KFold(n_splits=n_fold)
    preds = np.zeros(X.shape[0])

    for train_idx, valid_idx in folds.split(X, y):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model.fit(X_train, y_train)
        preds[valid_idx] = model.predict(X_valid).flatten()

    model.fit(X, y)
    test_preds = model.predict(X_test)

    return preds.reshape(-1, 1), test_preds.reshape(-1, 1)

In [41]:
from sklearn import datasets
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR

X, y = datasets.load_diabetes(return_X_y=True)
X = pd.DataFrame(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True)

base_models = [LinearRegression(), Lasso(), SVR()]
for model in base_models:
    GetPreds(model, X, y, X_test)